In [1]:
import math as m
import numpy as np
import matplotlib.pyplot as plt
from cvxpy import *

Data set

In [2]:
A=np.array([[0, 10, 5, 2], [10, 0, 7, 3], [5, 7, 0, 6], [2, 3, 6, 0]])
C_0=100
C_t=1
n=np.shape(A)[0]
maxval=A.max()+1

Objective function: $k \cdot C_0 + C_t \cdot TS$

In [3]:
x=np.array([0,0,0,1])
k=np.count_nonzero(x)
e=np.zeros(n)+1

2 part. Divide all vertices to clusters

In [4]:
def distances(A,x):
    n=np.size(x)
    #cl=np.arange(0,n)
    maxval=A.max()+1
    dist=(np.zeros(n)+maxval)*((x-1)*(-1))
    #print(dist)
    for i in range(n):
        if x[i]==0:
            for j in range(n):
                if A[i,j]>0 and x[j]==1 and dist[i]>A[i,j]:
                    dist[i]=A[i,j]
    #                cl[i]=j
#   print(dist)
#   print(cl)
    return dist

3 part. Compute TS for this clusters

In [5]:
def TotalCost(k,C_0,C_t,dist,e):
    return k*C_0+C_t*(2*e.dot(dist))
    

Markov chain choosing of k

In [6]:
def stationar(A):
    n=np.shape(A)[0]
    e=np.zeros(n)+1
    A_sum=np.outer(A.dot(e),e)
    P=A/(A_sum)
    P=P.T-np.eye(n)
    x = Variable(n)
    objective = Minimize(norm(P*x,2))
    constraints = [sum_entries(x) == 1,
                  x >= 0]
    prob = Problem(objective, constraints)
    result = prob.solve()
    return x.value

Heuristic algorithm with bad choosing of depots and bad TS

In [7]:
#A=np.array([[0, 10, 5, 2], [10, 0, 7, 3], [5, 7, 0, 6], [2, 3, 6, 0]])
#max_k=3
#C_0=100
#C_t=1
def Heuristic_1(A,max_k,C_0,C_t):
    n=np.shape(A)[0]
    maxval=A.max()+1
    e=np.zeros(n)+1
    x=np.zeros(n)
    Cost=np.zeros(n+1)+n*C_0
    k=0
    while k < max_k:
        m=0
        k=k+1
        x1=np.copy(x)
        while m < n:
            if x[m]==1:
                m=m+1
            else:
                x[m]=1
                dist=distances(A,x)
 #               print(dist)
                TotCost=k*C_0+C_t*(2*e.dot(dist))
#                print(TotCost)
                if TotCost<Cost[k-1]:
                    Cost[k]=TotCost
                    m=n
                else:
                    x[m]=0
                    m+=1
        if e.dot(x1-x)==0:
            k=max_k
    if e.dot(x)==0:
        x=x+1
    return np.min(Cost), np.argmin(Cost), x

#print(Heuristic_1(A,max_k,C_0,C_t))

In [9]:
A=np.array([[0, 10, 5, 2], [10, 0, 7, 3], [5, 7, 0, 6], [2, 3, 6, 0]])
max_k=2
C_0=100
C_t=1
n=10

def Heuristic_fast(A,max_k,C_0,C_t):
    n=np.shape(A)[0]
    maxval=A.max()+1
    e=np.zeros(n)+1
    x=np.zeros(n)
    Cost=np.zeros(n+1)+n*C_0
    k=0
    y=stationar(A)
    while k < max_k:
        k=k+1
        x1=np.copy(x)
        x[np.argmin(y)]=1
        y[np.argmin(y)]=maxval
        dist=distances(A,x)
#        print(dist)
        TotCost=k*C_0+C_t*(2*e.dot(dist))
#        print(TotCost)
        if TotCost<Cost[k-1]:
            Cost[k]=TotCost
        else:
            k=max_k
            x=x1
    if e.dot(x)==0:
        x=x+1
    return np.min(Cost), np.argmin(Cost), x

print(Heuristic_fast(A,max_k,C_0,C_t))

(122.0, 1, array([ 0.,  0.,  0.,  1.]))


In [10]:
def createA(n):
    coord=np.random.randint(0, 100, size=(n,2))
    A=np.zeros([n,n])
    for i in range(n):
        for j in range (n):
            A[i,j]=((coord[i,0]-coord[j,0])**2+(coord[i,1]-coord[j,1])**2)**(0.5)
            A[j,i]=A[i,j]
    return A

In [11]:
n=10
A=createA(n)
C_0=np.random.randint(100,1000)
C_t=np.random.randint(1,10)
max_k=n/2
print(C_0,C_t)
print(Heuristic_fast(A,max_k,C_0,C_t))
print(Heuristic_1(A,max_k,C_0,C_t))

795 6
(4817.0967510665596, 1, array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]))
(4776.9376700389803, 3, array([ 1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]))
